# Machine Learning Immediate Recurrence of Atrial Fibrillation (IRAF) Susceptibility

Immediate Recurrence of Atrial Fibrillation (IRAF) occurs after synchronized cardioversion with a prevalence of 5-26%. [[1]] Recurrence decreases cardioversion efficacy and puts patients through the unnecessary risk of unsuccessful cardioversion. Better knowledge of patient susceptibility factors will allow clinicians to pre-treat these patients to reduce IRAF incidence. This project aims to shed more light on IRAF susceptibility factors through analysis of a cardioversion database.

[1]: https://deepblue.lib.umich.edu/bitstream/handle/2027.42/72654/j.1540-8167.2003.02415.x.pdf?sequence=1&isAllowed=y

In [2]:
import pandas as pd

In [3]:
data_file = '../ml-iraf-data/cvdb_v0_CLEANED_sanp.xlsx'
cvdb = pd.read_excel(data_file)

In [4]:
print(list(cvdb))

['PID', 'Gender', 'DOB', 'Race', 'Expiration_Date', 'Date_Of_Service', 'Procedure_Type', 'Staff', 'Room', 'Patient_Height', 'Patient_Weight', 'BMI', 'BSA', 'Baseline_BP_Systolic', 'Baseline_BP_Diastolic', 'Baseline_Heart_Rate', 'Baseline_O2_Saturation', 'Score_Activity', 'Score_Respiration', 'Score_Consciousness', 'Score_O2_Saturation', 'Score_Ambulation', 'Score_Total', 'INR', 'INR_Date', 'Creatinine', 'Potassium', 'Caffeine', 'Alcohol', 'Smoking', 'Smoking_Packs_Per_Day', 'Smoking_Quit_Years_Ago', 'Smoking_How_Many_Years', 'Indications', 'NYHA_Class', 'Hx_AFib_Or_AFlutter', 'Coronary_Artery_Disease', 'Valvular_Heart_Disease', 'Cardiomyopathy', 'Cardiomyopathy_Type', 'Congenital_Heart_Disease', 'Hypertension', 'Diabetes', 'DVT_Or_PE', 'Stroke', 'TIA', 'PAD', 'Min_LVEF', 'Min_LVEF_Modality', 'Max_LVEF', 'Max_LVEF_Modality', 'Has_Pacemaker', 'Pacemaker_Type', 'Has_ICD', 'ICD_Type', 'Prior_Cardioversion', 'Prior_AFib_Ablation', 'Prior_AFlutter_ATach_Ablation', 'Med_ACE_Inhibitor', 'Med_A

In [5]:
print(cvdb.shape)

(3670, 274)


## Begin Cleaning Data

In [31]:
cvdb.columns.get_loc('Success_1')

143

In [11]:
print(pd.unique(cvdb['Success_1']))

['Transiently successful' 'Successful' nan 'Unsuccessful']


In [12]:
print(pd.unique(cvdb['Attempted_1']))

['Yes' nan]


In [13]:
cvdb = cvdb[cvdb['Attempted_1'] == 'Yes']

In [15]:
print(pd.unique(cvdb['Success_1']))

['Transiently successful' 'Successful' nan 'Unsuccessful']


In [49]:
#print(cvdb[cvdb['Success_1'].isnull()]['Shock_Type_1'])
#subset = [a and b for a, b in zip(cvdb['Success_1'].isnull(), cvdb['Shock_Type_1'].isnull())]
#cvdb[subset].iloc[:,140:150]

In [53]:
subset = [a and b for a, b in zip(cvdb['Success_1'].isnull(), cvdb['Shock_Type_1'].isnull())]
notsubset = [not i for i in subset]
cvdb = cvdb[notsubset]

In [54]:
print(cvdb.shape)

(3608, 274)
